In [ ]:
from trulens_eval.quicktest.quicktest import QuickTest

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
# Imports from langchain to build app
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("hi")

In [ ]:
from trulens_eval.quicktest.quicktest import QuickTest
from trulens_eval import TruChain

from trulens_eval import OpenAI
openai_provider = OpenAI()

In [ ]:
quicktest = QuickTest(
    app_callable = rag_chain.invoke,
    feedback_provider = openai_provider,
    eval_framework = TruChain
    )

In [ ]:
test_cases = quicktest.generate_hallucination_test_cases(number_test_cases=5)

In [ ]:
feedbacks = quicktest.get_rag_triad()

In [ ]:
recorder = quicktest.get_recorder(feedbacks = feedbacks)

In [ ]:
with recorder as recording:
    for test_case in test_cases:
        rag_chain.invoke(test_case)

In [ ]:
tru.get_leaderboard(app_ids=[])

In [ ]:
quicktest.evaluate_hallucination(number_test_cases=10)

In [ ]:
from trulens_eval import Tru

tru = Tru()

tru.get_leaderboard(app_ids=[])

In [ ]:
tru.run_dashboard()